# Reference
- namedtuple
https://stackoverflow.com/questions/2970608/what-are-named-tuples-in-python
```python
from collections import namedtuple
Point = namedtuple('Point', 'x y')
pt1 = Point(1.0, 5.0)
pt2 = Point(2.5, 1.5)

from math import sqrt
line_length = sqrt((pt1.x-pt2.x)**2 + (pt1.y-pt2.y)**2)
```

# stack vs cat

In [1]:
import torch

In [2]:
t1 = torch.tensor([1,1,1])
t2 = torch.tensor([2,2,2])
t3 = torch.tensor([3,3,3])

In [3]:
t1.shape

torch.Size([3])

In [4]:
t1.unsqueeze(0).shape

torch.Size([1, 3])

In [5]:
cat1 = torch.cat((t1,t2,t3), dim=0)  

In [6]:
cat1.shape   # same axis

torch.Size([9])

In [7]:
stack1 = torch.stack((t1, t2, t3), dim=0)

In [8]:
stack1.shape   # new axis

torch.Size([3, 3])

In [9]:
torch.cat(
    (
        t1.unsqueeze(0),
        t2.unsqueeze(0),
        t3.unsqueeze(0)
    ),
    dim=0
)

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [10]:
torch.cat(
    (
        t1.unsqueeze(0),
        t2.unsqueeze(0),
        t3.unsqueeze(0)
    ),
    dim=0
).shape

torch.Size([3, 3])

In [11]:
torch.stack(
    (t1, t2, t3),
    dim=0
)

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [12]:
torch.stack(
    (t1, t2, t3),
    dim=1
)

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

# Run builder

In [13]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [14]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple("testRun", params.keys())
        
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [15]:
params = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000]
)

In [16]:
print(params)

OrderedDict([('lr', [0.01, 0.001]), ('batch_size', [1000, 10000])])


In [17]:
a = []
c = []
b = namedtuple("test", params.keys())
for each in product(*params.values()):
    print(each)
    print(b(*each))

(0.01, 1000)
test(lr=0.01, batch_size=1000)
(0.01, 10000)
test(lr=0.01, batch_size=10000)
(0.001, 1000)
test(lr=0.001, batch_size=1000)
(0.001, 10000)
test(lr=0.001, batch_size=10000)


In [18]:
runs = RunBuilder.get_runs(params)
runs

[testRun(lr=0.01, batch_size=1000),
 testRun(lr=0.01, batch_size=10000),
 testRun(lr=0.001, batch_size=1000),
 testRun(lr=0.001, batch_size=10000)]

In [19]:
for i in runs:
    for j in i: 
        print(j)

0.01
1000
0.01
10000
0.001
1000
0.001
10000


In [20]:
run = runs[0]
run

testRun(lr=0.01, batch_size=1000)

In [21]:
print(run.lr)

0.01


In [22]:
for each in runs:
    print(run, run.lr, run.batch_size)

testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000


In [23]:
# another example
params_gpu_cpu = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000],
    device=['cuda', 'cpu']
)

In [24]:
runs = RunBuilder.get_runs(params_gpu_cpu)
runs

[testRun(lr=0.01, batch_size=1000, device='cuda'),
 testRun(lr=0.01, batch_size=1000, device='cpu'),
 testRun(lr=0.01, batch_size=10000, device='cuda'),
 testRun(lr=0.01, batch_size=10000, device='cpu'),
 testRun(lr=0.001, batch_size=1000, device='cuda'),
 testRun(lr=0.001, batch_size=1000, device='cpu'),
 testRun(lr=0.001, batch_size=10000, device='cuda'),
 testRun(lr=0.001, batch_size=10000, device='cpu')]

### How to build the RunBuilder

In [25]:
params = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000]
)

In [26]:
params.keys()

odict_keys(['lr', 'batch_size'])

In [27]:
params.values()

odict_values([[0.01, 0.001], [1000, 10000]])

In [28]:
Run = namedtuple("Run", params.keys())

In [29]:
runs = []
for v in product(*params.values()):
    runs.append(Run(*v))
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [30]:
for run in RunBuilder.get_runs(params):
    comment = f"-{run}"
    print(f"comment: {comment} lr={run.lr}, batch_size={run.batch_size}")

comment: -testRun(lr=0.01, batch_size=1000) lr=0.01, batch_size=1000
comment: -testRun(lr=0.01, batch_size=10000) lr=0.01, batch_size=10000
comment: -testRun(lr=0.001, batch_size=1000) lr=0.001, batch_size=1000
comment: -testRun(lr=0.001, batch_size=10000) lr=0.001, batch_size=10000


# CNN example with paramters tuning

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [32]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output

In [33]:
import pandas as pd
import time
import json

In [34]:
from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [35]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=6,
                               kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, 
                               out_channels=12,
                               kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4,
                             out_features=120)
        self.fc2 = nn.Linear(in_features=120,
                             out_features=60)
        self.out = nn.Linear(in_features=60,
                             out_features=10)
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [36]:
# flatten example

t = torch.tensor([[[1, 2],
                       [3, 4]],
                      [[5, 6],
                       [7, 8]]])

In [37]:
t

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [38]:
t.shape

torch.Size([2, 2, 2])

In [39]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple("Run", params.keys())
        
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

In [40]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(
             self.network
            ,images.to(getattr(run, 'device', 'cpu'))
        )
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0   

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
                
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item() * batch[0].shape[0]
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
            self.run_data
            ,orient='columns'
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [41]:
train_set = torchvision.datasets.FashionMNIST(
    root = "./data",
    train = True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [42]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    shuffle=[True]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    
    m.begin_run(run, network, loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            
            images, labels = batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)  
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.01771,0.607333,5.688169,6.351777,0.01,1000,True


In [43]:
# Pytorch dataloader num_workers

params = OrderedDict(
    lr = [.01],
    batch_size =[100], # 1000, 10000
    num_workers = [0], # 1,2,4,8,16
    # shuffle=[True, False]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set, 
                        batch_size=run.batch_size, 
                        num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            images, labels = batch
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers
0,1,1,0.592946,0.775633,6.328691,6.404869,0.01,100,0


# Pytorch and GPU

In [44]:

t = torch.ones(1,1,28, 28)
t

tensor([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [45]:
network=Network()
network

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [46]:
t = t.cuda()
network = network.cuda()

In [47]:
gpu_pred = network(t)
gpu_pred.device

/home/loveplay1983/Workstation/Anaconda/anaconda/envs/torch/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


device(type='cuda', index=0)

In [48]:
# moving to cpu
t = t.cpu()
network = network.cpu()

In [49]:
cpu_pred = network(t)
cpu_pred.device

device(type='cpu')

In [50]:
t1 = torch.tensor([
    [1,2],
    [3,4]
])

t2 = torch.tensor([
    [5,6],
    [7,8]
])

In [51]:
t1.device, t2.device

(device(type='cpu'), device(type='cpu'))

In [52]:
t1 = t1.to('cuda')
t1.device

device(type='cuda', index=0)

In [53]:
try: t1+t2
    
except Exception as e: print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


In [54]:
try: t2+t1
except Exception as e: print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


In [55]:
t2 = t2.to("cuda")
t1+t2

tensor([[ 6,  8],
        [10, 12]], device='cuda:0')

In [56]:
torch.cuda.is_available()

True

In [57]:
# Using GPU
params = OrderedDict(
    lr = [.01],
    batch_size = [20000],
    num_workers = [1],
    device = ["cuda", "cpu"]
)


m = RunManager()

for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    network = Network().to(device)
    loader = DataLoader(train_set, \
                        batch_size=run.batch_size, 
                        num_workers=run.num_workers)
    optimizer=optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device
0,1,1,2.282136,0.114333,4.107421,11.373373,0.01,20000,1,cuda
1,2,1,2.283626,0.137500,4.995587,13.575877,0.01,20000,1,cpu


In [58]:
pd.DataFrame.from_dict(m.run_data).sort_values("epoch duration")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device
0,1,1,2.282136,0.114333,4.107421,11.373373,0.01,20000,1,cuda
1,2,1,2.283626,0.137500,4.995587,13.575877,0.01,20000,1,cpu


# Data normalization

In [59]:
loader = DataLoader(train_set, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(0.2860), tensor(0.3530))

In [60]:
loader = DataLoader(train_set, batch_size=1000, num_workers=1)
num_of_pixels = len(train_set) * 28 * 28

total_sum = 0
for batch in loader: 
    total_sum += batch[0].sum()    
mean = total_sum / num_of_pixels

sum_of_squared_error = 0
for batch in loader: sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)

mean, std

(tensor(0.2860), tensor(0.3530))

In [61]:
import matplotlib.pyplot as plt

In [62]:
# plt.hist(data[0].flatten())
# plt.axvline(data[0].mean())

In [63]:
# Using mean and std

train_set_normal = torchvision.datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [64]:
loader = DataLoader(train_set_normal, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
data[0].mean(), data[0].std()

(tensor(-1.8838e-07), tensor(1.))

In [65]:
# plt.hist(data[0].flatten())
# plt.axvline(data[0].mean())

In [66]:
trainsets = {
    "not_normal": train_set,
    "normal": train_set_normal
}

In [67]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    num_workers = [1],
    device = ["cuda"],
    trainset = ["not_normal", "normal"]
)

In [68]:
m = RunManager()

for run in RunBuilder.get_runs(params):
    
    device = torch.device(run.device)
    network = Network().to(device)
    loader = DataLoader(trainsets[run.trainset], 
                        batch_size=run.batch_size,
                        num_workers=run.num_workers)
    optimizer=optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    
    for epoch in range(20):
        
        m.begin_epoch()
        
        for batch in loader:
            
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images)
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save("results")

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
0,1,1,1.012159,0.611050,3.610131,4.217925,0.01,1000,1,cuda,not_normal
1,1,2,0.549667,0.791583,3.815608,8.085014,0.01,1000,1,cuda,not_normal
2,1,3,0.448199,0.835700,3.796827,11.933259,0.01,1000,1,cuda,not_normal
3,1,4,0.397315,0.852417,3.597420,15.581885,0.01,1000,1,cuda,not_normal
4,1,5,0.360743,0.864300,3.581840,19.217166,0.01,1000,1,cuda,not_normal
5,1,6,0.334781,0.875033,3.766021,23.031758,0.01,1000,1,cuda,not_normal
6,1,7,0.317988,0.881183,3.802334,26.885738,0.01,1000,1,cuda,not_normal
7,1,8,0.303748,0.886667,3.719511,30.656826,0.01,1000,1,cuda,not_normal
8,1,9,0.292321,0.890867,3.541994,34.252670,0.01,1000,1,cuda,not_normal
9,1,10,0.279175,0.896267,3.515964,37.821379,0.01,1000,1,cuda,not_normal


In [69]:
pd.DataFrame.from_dict(m.run_data).sort_values('accuracy', ascending=False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
39,2,20,0.220608,0.916133,6.216390,121.004527,0.01,1000,1,cuda,normal
19,1,20,0.227543,0.915600,3.646927,75.164638,0.01,1000,1,cuda,not_normal
18,1,19,0.231809,0.913867,3.678926,71.468807,0.01,1000,1,cuda,not_normal
38,2,19,0.229693,0.913183,5.932316,114.732335,0.01,1000,1,cuda,normal
16,1,17,0.237322,0.912200,3.649055,64.029450,0.01,1000,1,cuda,not_normal
35,2,16,0.234162,0.912050,5.941553,96.523380,0.01,1000,1,cuda,normal
17,1,18,0.236283,0.911567,3.651999,67.734567,0.01,1000,1,cuda,not_normal
34,2,15,0.238303,0.911050,5.903754,90.526949,0.01,1000,1,cuda,normal
36,2,17,0.235082,0.910967,6.021453,102.599032,0.01,1000,1,cuda,normal
32,2,13,0.239388,0.910817,5.902164,78.634251,0.01,1000,1,cuda,normal


In [70]:
# torch.manual_seed(50)
# sequential1 = nn.Sequential(
#       nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#     , nn.ReLU()
#     , nn.MaxPool2d(kernel_size=2, stride=2)
#     , nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
#     , nn.ReLU()
#     , nn.MaxPool2d(kernel_size=2, stride=2)
#     , nn.Flatten(start_dim=1)  
#     , nn.Linear(in_features=12*4*4, out_features=120)
#     , nn.ReLU()
#     , nn.Linear(in_features=120, out_features=60)
#     , nn.ReLU()
#     , nn.Linear(in_features=60, out_features=10)
# )


# torch.manual_seed(50)
# layers = OrderedDict([
#      ('conv1', nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5))
#     ,('relu1', nn.ReLU())
#     ,('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
    
#     ,('conv2', nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5))
#     ,('relu2', nn.ReLU())
#     ,('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
    
#     ,('flatten', nn.Flatten(start_dim=1)  )
#     ,('fc1', nn.Linear(in_features=12*4*4, out_features=120))
#     ,('relu3', nn.ReLU())
    
#     ,('fc2', nn.Linear(in_features=120, out_features=60))
#     ,('relu4', nn.ReLU())
#     ,('out', nn.Linear(in_features=60, out_features=10))
# ])

# sequential2 = nn.Sequential(layers)


# torch.manual_seed(50)
# sequential3 = nn.Sequential()
# sequential3.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5))
# sequential3.add_module('relu1', nn.ReLU())
# sequential3.add_module('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
    
# sequential3.add_module('conv2', nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5))
# sequential3.add_module('relu2', nn.ReLU())
# sequential3.add_module('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
    
# sequential3.add_module('flatten', nn.Flatten(start_dim=1))
# sequential3.add_module('fc1', nn.Linear(in_features=12*4*4, out_features=120))
# sequential3.add_module('relu3', nn.ReLU())
    
# sequential3.add_module('fc2', nn.Linear(in_features=120, out_features=60))
# sequential3.add_module('relu4', nn.ReLU())
# sequential3.add_module('out', nn.Linear(in_features=60, out_features=10))

# RunManager updated

In [71]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn. functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output

import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [72]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple("Run", params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [73]:
class RunManager():
    def __init__(self):
        # Run
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        # model, dataloader, and tensorboard
        self.network = None
        self.loader = None
        self.tb = None
        
        # Epoch
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
    def begin_run(self, run, network, loader):
        self.run_start_time = time.time()
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
#         self.tb = SummaryWriter(comment=f"-{run}")
        self.tb = SummaryWriter()
    
        # Load the images, and labels
        imgs, lbls = next(iter(self.loader))
        grid = torchvision.utils.make_grid(imgs)
        
#         self.tb.add_image("images", grid)
#         self.tb.add_graph(
#             self.network,
#             imgs.to(getattr(run, "device", "cpu"))
#         )
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        # for each epoch, all the losses divided by the len(dataset)
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar("Loss", loss, self.epoch_count)
        self.tb.add_scalar("Accuracy", accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f"{name}.grad", param.grad, self.epoch_count)
            
        # The output result collection
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k, v in self.run_params._asdict().items():
            results[k] = v
        self.run_data.append(results)
        
        # Define the output result from dict
        df = pd.DataFrame.from_dict(self.run_data, orient="columns")
    
        # Clear the canvas and display teh output result
        clear_output(wait=True)
        display(df)
        
    def end_run(self):
        """
        for the run is ended, tensorboard, and epoch count gets reset
        """
        self.tb.close()
        self.epoch_count = 0
            
    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item() * batch[0].shape[0]
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
        
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        # CSV
        pd.DataFrame.from_dict(
            self.run_data,
            orient="columns"
        ).to_csv(f"{fileName}.csv")
        
        # JSON 
        with open(f"{fileName}.json", "w", encoding="utf-8") as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

# Batch Norm


In [74]:
# No BatchNorm
torch.manual_seed(50)
network_nobatch = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.Flatten(start_dim=1)  
    , nn.Linear(in_features=12*4*4, out_features=120)
    , nn.ReLU()
    , nn.Linear(in_features=120, out_features=60)
    , nn.ReLU()
    , nn.Linear(in_features=60, out_features=10)
)

In [75]:
# BatchNorm
torch.manual_seed(50)
network_batch = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.BatchNorm2d(6)
    , nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.Flatten(start_dim=1)  
    , nn.Linear(in_features=12*4*4, out_features=120)
    , nn.ReLU()
    , nn.BatchNorm1d(120)
    , nn.Linear(in_features=120, out_features=60)
    , nn.ReLU()
    , nn.Linear(in_features=60, out_features=10)
)

In [76]:
# trainset
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [77]:
# get mean, std
loader = DataLoader(train_set, batch_size=len(train_set), num_workers=1)
data = next(iter(loader))
mean = data[0].mean()
std = data[0].std()
mean, std

(tensor(0.2860), tensor(0.3530))

In [78]:
# generate trainset with normal distribution
train_set_normal = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
        ,transforms.Normalize(mean, std)
    ])
)

In [79]:
# different trainsets
trainsets = {
    'not_normal': train_set
    ,'normal': train_set_normal
}

In [80]:
# different networks
networks = {
    'no_batch_norm': network_nobatch
    ,'batch_norm': network_batch
}

In [81]:
params = OrderedDict(
    lr = [.01]
    , batch_size = [1000]
    , num_workers = [1]
    , device = ['cuda']
    , trainset = ['normal']
    , network = list(networks.keys())
)
m = RunManager()
for run in RunBuilder.get_runs(params):

    device = torch.device(run.device)
    network = networks[run.network].to(device)
    loader = DataLoader(trainsets[run.trainset], batch_size=run.batch_size, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset,network
0,1,1,0.910890,0.661767,6.495939,6.884146,0.01,1000,1,cuda,normal,no_batch_norm
1,1,2,0.482729,0.816633,6.212908,13.149975,0.01,1000,1,cuda,normal,no_batch_norm
2,1,3,0.401896,0.851133,5.971106,19.175631,0.01,1000,1,cuda,normal,no_batch_norm
3,1,4,0.355195,0.869033,5.980349,25.211472,0.01,1000,1,cuda,normal,no_batch_norm
4,1,5,0.332257,0.877983,5.906584,31.183192,0.01,1000,1,cuda,normal,no_batch_norm
5,2,1,0.582896,0.788033,6.070597,6.450924,0.01,1000,1,cuda,normal,batch_norm
6,2,2,0.351707,0.867883,5.896474,12.419596,0.01,1000,1,cuda,normal,batch_norm
7,2,3,0.310723,0.882883,5.936544,18.424352,0.01,1000,1,cuda,normal,batch_norm
8,2,4,0.286737,0.891650,5.807678,24.306875,0.01,1000,1,cuda,normal,batch_norm
9,2,5,0.267775,0.898567,5.733323,30.109798,0.01,1000,1,cuda,normal,batch_norm


In [82]:
pd.DataFrame.from_dict(m.run_data).sort_values('accuracy', ascending=False)

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset,network
9,2,5,0.267775,0.898567,5.733323,30.109798,0.01,1000,1,cuda,normal,batch_norm
8,2,4,0.286737,0.891650,5.807678,24.306875,0.01,1000,1,cuda,normal,batch_norm
7,2,3,0.310723,0.882883,5.936544,18.424352,0.01,1000,1,cuda,normal,batch_norm
4,1,5,0.332257,0.877983,5.906584,31.183192,0.01,1000,1,cuda,normal,no_batch_norm
3,1,4,0.355195,0.869033,5.980349,25.211472,0.01,1000,1,cuda,normal,no_batch_norm
6,2,2,0.351707,0.867883,5.896474,12.419596,0.01,1000,1,cuda,normal,batch_norm
2,1,3,0.401896,0.851133,5.971106,19.175631,0.01,1000,1,cuda,normal,no_batch_norm
1,1,2,0.482729,0.816633,6.212908,13.149975,0.01,1000,1,cuda,normal,no_batch_norm
5,2,1,0.582896,0.788033,6.070597,6.450924,0.01,1000,1,cuda,normal,batch_norm
0,1,1,0.910890,0.661767,6.495939,6.884146,0.01,1000,1,cuda,normal,no_batch_norm


# Resetting weights

In [83]:
torch.manual_seed(50)
layer = nn.Linear(2,1)

In [84]:
layer.weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

In [85]:
t = torch.rand(2)
o = layer(t)
o.backward()

In [86]:
optimizer = optim.Adam(layer.parameters(), lr=.01)
optimizer.step()

In [87]:
layer.weight

Parameter containing:
tensor([[ 0.1569, -0.6200]], requires_grad=True)

In [88]:
torch.manual_seed(50)
layer.reset_parameters()   # Reset

In [89]:
layer.weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

### # Individual layer inside a network

In [90]:

torch.manual_seed(50)
network = nn.Sequential(nn.Linear(2,1))
network[0]

Linear(in_features=2, out_features=1, bias=True)

In [91]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

In [92]:
t = torch.rand(2)
o = network(t)
o.backward()

optimizer = optim.Adam(network.parameters(), lr=.01)
optimizer.step()

In [93]:
network[0].weight

Parameter containing:
tensor([[ 0.1569, -0.6200]], requires_grad=True)

In [94]:
torch.manual_seed(50)
network[0].reset_parameters()

In [95]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

### All weights layer by layer

In [96]:
network = nn.Sequential(nn.Linear(2,1))
network[0].weight

Parameter containing:
tensor([[-0.6500, -0.1395]], requires_grad=True)

In [97]:
torch.manual_seed(50)
for module in network.children():
    module.reset_parameters()

In [98]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

In [99]:
network = nn.Sequential(
    nn.Linear(2,1),
    nn.Softmax()
)

In [100]:
network

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Softmax(dim=None)
)

In [101]:
try:
    torch.manual_seed(50)
    for module in network.children():
        module.reset_parameters()
except Exception as E:
    print(E)

'Softmax' object has no attribute 'reset_parameters'


### All weights using snapshot

In [102]:
torch.manual_seed(50)
network = nn.Sequential(nn.Linear(2,1))

In [103]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

In [104]:
torch.save(network.state_dict(), "./network.pt")

In [105]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

### Weights re-initialization

In [106]:
torch.manual_seed(50)
network = nn.Sequential(nn.Linear(2,1))

In [107]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

In [108]:
t= torch.rand(2)
o = network(t)
o.backward()

In [109]:
optimizer = optim.Adam(network.parameters(), lr=.01)
optimizer.step()

In [110]:
network[0].weight

Parameter containing:
tensor([[ 0.1569, -0.6200]], requires_grad=True)

In [111]:
# reinitialization
torch.manual_seed(50)
network = nn.Sequential(nn.Linear(2,1))

In [112]:
network[0].weight

Parameter containing:
tensor([[ 0.1669, -0.6100]], requires_grad=True)

# Multiple Networks

In [113]:
class NetworkFactory():
    @staticmethod
    def get_network(name):
        if name == "network":
            torch.manual_seed(50)
            return nn.Sequential(
                  nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
                , nn.ReLU()
                , nn.MaxPool2d(kernel_size=2, stride=2)
                , nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
                , nn.ReLU()
                , nn.MaxPool2d(kernel_size=2, stride=2)
                , nn.Flatten(start_dim=1)  
                , nn.Linear(in_features=12*4*4, out_features=120)
                , nn.ReLU()
                , nn.Linear(in_features=120, out_features=60)
                , nn.ReLU()
                , nn.Linear(in_features=60, out_features=10)
            )
        else:
            return None

In [114]:
params = OrderedDict(
    lr = [.01, .001]
    , batch_size = [1000]
    , num_workers = [1]
    , device = ['cuda']
    , network = ['network']
)
m = RunManager()
for run in RunBuilder.get_runs(params):

    device = torch.device(run.device)
    network = NetworkFactory.get_network(run.network).to(device)
    loader = DataLoader(train_set, batch_size=run.batch_size, num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0].to(device)
            labels = batch[1].to(device)
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,network
0,1,1,1.026914,0.606883,3.646528,3.949945,0.010,1000,1,cuda,network
1,1,2,0.554524,0.781700,3.653482,7.664266,0.010,1000,1,cuda,network
2,1,3,0.461596,0.830450,3.712839,11.432393,0.010,1000,1,cuda,network
3,1,4,0.408623,0.851517,3.714561,15.196077,0.010,1000,1,cuda,network
4,1,5,0.374598,0.863033,3.715599,18.988585,0.010,1000,1,cuda,network
5,2,1,1.562656,0.497350,3.516679,3.819836,0.001,1000,1,cuda,network
6,2,2,0.736826,0.714400,3.540333,7.422279,0.001,1000,1,cuda,network
7,2,3,0.634179,0.752350,3.430017,10.906043,0.001,1000,1,cuda,network
8,2,4,0.578988,0.776950,3.530773,14.486950,0.001,1000,1,cuda,network
9,2,5,0.539403,0.794267,3.413850,17.949183,0.001,1000,1,cuda,network


# PyTorch Hooks

In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [116]:
import matplotlib.pyplot as plt

In [117]:
feature_maps = []
network1 = Network()

In [118]:
network1

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)